In [2]:
# set the environment to simulate the program
# transformers is installed to perform tasks on different modalities such as text, vision, and audio.(kogpt is the dataset of text.)
!pip install tensorflow==2.10.0
!pip install transformers

In [3]:
# import the packages
import tensorflow as tf
# use it for the iteration based operations.
import tqdm
# Importing necessary libraries for Natural Language Processing (NLP) tasks.
from transformers import AutoTokenizer
# TFGPT2LMHeadModel: Another module from Hugging Face Transformers, specifically the GPT-2 language model
from transformers import TFGPT2LMHeadModel

In [4]:
# assign the tokenizer and the model
### START CODE HERE ###

# find & assign tokenizer and model; 'skt/kogpt2-base-v2'
# load the tokenizer for the pre-trained GPT-2 model identified by 'skt/kogpt2-base-v2'
# the parameters bos_token, eos_token, and pad_token specify special tokens denoting the beginning, end, and padding of a sentence, respectively
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
# the from_pt=True parameter indicates the loading of weights trained in PyTorch into a TensorFlow model.
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

### END CODE HERE ###

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [5]:
# print the result of the summary about the model
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 125164032 
 r)                                                              
                                                                 
Total params: 125,164,032
Trainable params: 125,164,032
Non-trainable params: 0
_________________________________________________________________


In [6]:
# print the information of the dataset to check it is right
print(tokenizer.bos_token_id) # prints the ID of the special token representing the start of a sentence.
print(tokenizer.eos_token_id) # prints the ID of the special token representing the end of a sentence.
print(tokenizer.pad_token_id) # prints the ID of the special token representing padding.
print('-' * 10)
# the tokenizer.decode() function decodes IDs into their corresponding strings.
print(tokenizer.decode(1))  # decodes the ID corresponding to bos_token_id into the string representing the start of a sentence.
print(tokenizer.decode(2))  # decodes the ID corresponding to eos_token_id into the string representing the end of a sentence.
print(tokenizer.decode(3))  # decodes the token at ID 3.
print(tokenizer.decode(4))  # decodes the token at ID 4.

1
1
3
----------
</s>
<usr>
<pad>
<sys>


In [7]:
# Importing libraries for data handling and web operations.

# pandas (imported as pd): A powerful library for data manipulation and analysis in Python.
# It provides data structures like DataFrames to efficiently work with structured data, perform data cleaning,
# aggregation, filtering, and much more.
import pandas as pd
# urllib.request: A module in Python's standard library used for opening and reading URLs.(it's for load the dataset)
import urllib.request

In [8]:
# Using 'pd.read_csv()' to read the downloaded CSV file ('ChatBotData.csv') and load it into a pandas DataFrame named 'train_data'.
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
                           filename="ChatBotData.csv")
# read and assign the data to 'train_data'
train_data = pd.read_csv('ChatBotData.csv')

In [9]:
# display and check it is right assigned
display(train_data)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [10]:
# batch_size is assignd to 32
batch_size = 32

In [11]:
# define the function 'get_chat_data()'
def get_chat_data():
  # Iterate through pairs of questions and answers in the 'train_data' DataFrame
    for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
        bos_token = [tokenizer.bos_token_id]  # Initialize a list with the beginning-of-sequence token ID
        eos_token = [tokenizer.eos_token_id]  # Initialize a list with the end-of-sequence token ID
        # Encode the conversation context using the tokenizer:
        # '<usr>' signifies the user input, '<sys>' signifies the system response
        sent = tokenizer.encode('<usr>' + question + '<sys>' + answer)
        # Yield the tokens representing the conversation, including special tokens for start and end sequences
        yield bos_token + sent + eos_token

In [12]:
# Create a TensorFlow dataset using the 'from_generator' method, specifying 'get_chat_data' as the generator function.
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)
# Convert the generated dataset into a padded batch dataset using 'padded_batch'
# each parameters :batch_size is the size of each batch,padded_shapes are the elements of the dataset
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [13]:
# print out the batch[0] for checking the batch
for batch in dataset:
    print(batch[0])
    break

tf.Tensor(
[    1     2  9349  7888   739  7318   376     4 12557  6824  9108  9028
  7098 25856     1     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3], shape=(30,), dtype=int32)


In [14]:
# decode the batch by using tokenizer and print out the result.
str = tokenizer.decode(batch[0])
print(str)

</s><usr> 12시 땡!<sys> 하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [15]:
# encode the batch by using tokenizer and print out the result.
print(tokenizer.encode(str))

[1, 2, 9349, 7888, 739, 7318, 376, 4, 12557, 6824, 9108, 9028, 7098, 25856, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [16]:
# use the optimizer 'adam'
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [17]:
# calculate the steps and print out the result.
steps = len(train_data) // batch_size + 1
print(steps)

370


In [18]:
# Training loop that iterates through  a specified number of epochs and trains the model using gradient descent for the neural network model
EPOCHS = 3  # Number of training epochs

# Iterate through each epoch
for epoch in range(EPOCHS):
  epoch_loss = 0  # Initialize the loss for the current epoch
  # Iterate through each batch in the dataset (progress bar tracks steps)
  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape:
          # Forward pass: obtain model predictions for the input batch
          result = model(batch, labels=batch)
          loss = result[0]  # Extract the loss from the model output
          batch_loss = tf.reduce_mean(loss) # Compute the mean loss for the batch
      # Calculate gradients of the batch loss with respect to model trainable variables
      grads = tape.gradient(batch_loss, model.trainable_variables)
      # Update model weights using the optimizer (Adam optimizer in this case)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      # Aggregate the batch loss to compute the epoch loss
      epoch_loss += batch_loss / steps  # Accumulate batch loss for the epoch
  # Print the average loss for the current epoch
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

<ipython-input-18-9054695df6d8>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total=steps):


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.12707782


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.69820356


  0%|          | 0/370 [00:00<?, ?it/s]

[Epoch:    3] cost = 1.37539315


In [19]:
# Input text to be used in the conversation
text = '오늘도 좋은 하루!'

In [20]:
# Formatting the input text to simulate a conversation
sent = '<usr>' + text + '<sys>'

In [21]:
# Encode the formatted text using the tokenizer: beginning-of-sequence token ID + encodes the formatted text into token IDs.
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
# Convert the list of token IDs into a TensorFlow tensor
input_ids = tf.convert_to_tensor([input_ids])

In [22]:
# Generate text using the pre-trained model:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [23]:
# Decoding the generated output
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())

In [24]:
# Extracting and cleaning the system response
decoded_sentence.split('<sys> ')[1].replace('</s>', '')

'오늘도 좋은 하루네요.'

In [25]:
# Generating text using the model
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
# Decoding the newly generated output
tokenizer.decode(output[0].numpy().tolist())

'</s><usr> 오늘도 좋은 하루!<sys> 하루도 행복했으면 좋겠네요.</s>'

In [26]:
# define the function takes the user's text as input, formats it, generates a response using a language model,
# and returns the generated response to generate a response by the chatbot model

def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'  # Formatting the user's text
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent) # Encoding the formatted text
  input_ids = tf.convert_to_tensor([input_ids]) # Converting the token IDs to a TensorFlow tensor
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20) # Generating a response
  sentence = tokenizer.decode(output[0].numpy().tolist()) # Decoding the generated output
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')  # Extracting the system response and removing tags and special tokens
  return chatbot_response

In [27]:
# Returning the chatbot's response based on the user's input
return_answer_by_chatbot('안녕! 반가워~')

'축하해주신 모든 분들게 감사합니다.'

In [28]:
# Returning the chatbot's response based on the user's input
return_answer_by_chatbot('너는 누구야?')

'다가가 보세요.'

In [29]:
# Returning the chatbot's response based on the user's input
return_answer_by_chatbot('나랑 영화보자')

'같이 보고 싶은 마음이라면 먼저 하세요.'

In [30]:
# Returning the chatbot's response based on the user's input
return_answer_by_chatbot('너무 심심한데 나랑 놀자')

'잘 지낼 것 같네요.'

In [31]:
# Returning the chatbot's response based on the user's input
return_answer_by_chatbot('영화 해리포터 재밌어?')

'영화도 로맨스가 있죠.'

In [32]:
# Returning the chatbot's response based on the user's input
return_answer_by_chatbot('너 딥 러닝 잘해?')

'딥 러닝은 할 수 있지만 다른 걸 하게 되죠.'

In [33]:
# Returning the chatbot's response based on the user's input
return_answer_by_chatbot('커피 한 잔 할까?')

'커피 한 잔이랑 커피 한 잔이랑 커피 한 잔이 좋을 것 같습니다.'